In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
data=pd.read_csv("Datasets/emails.csv")
data.head(10)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [9]:
data['spam'].value_counts()

0    4360
1    1368
Name: spam, dtype: int64

In [37]:
stopwords=nltk.corpus.stopwords.words('english')
ps=nltk.PorterStemmer()
def count_punc(text):
    count=sum(1 for i in text if i in string.punctuation)
    return round(count/(len(text)-text.count(" ")),3)*100
data['body_len']=data['text'].apply(lambda x:len(x)-x.count(" "))
data['punct']=data['text'].apply(lambda x:count_punc(x))
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


In [27]:
from sklearn.model_selection import train_test_split
X=data[['text','body_len','punct']]
Y=data['spam']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,random_state=42)

,text,body_len,punct
2859,"Subject: interview on friday , may 5 th good ...",364,5.2
4130,Subject: re : time keeping please arrange for...,90,5.6
1137,Subject: you can win an extreme bedroom makeov...,514,7.6
3161,Subject: meeting confirmed : mit / aa new valu...,1702,10.0
5286,"Subject: grades pam , the term papers arrive...",365,2.7
...,...,...,...
3772,Subject: john sherriff ' s copper position te...,780,12.7
5191,Subject: the national forum on corporate finan...,932,7.3
5226,"Subject: re : my first draft quentin , i for...",227,2.6
5390,Subject: why johan dahl and the mri energy sta...,592,5.7


In [49]:
vect=TfidfVectorizer(analyzer=clean_text)
vect_fit=vect.fit(x_train['text'])
tf_train=vect.transform(x_train['text'])
tf_test=vect.transform(x_test['text'])
tf_train_vect=pd.concat([x_train[['body_len','punct']].reset_index(drop=True),pd.DataFrame(tf_train.toarray())],axis=1)
tf_test_vect=pd.concat([x_test[['body_len','punct']].reset_index(drop=True),pd.DataFrame(tf_test.toarray())],axis=1)
tf_train_vect.head()

,body_len,punct,0,1,2,3,4,5,6,7,...,22771,22772,22773,22774,22775,22776,22777,22778,22779,22780
0,364,5.2,0.000000,0.0,0.083283,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90,5.6,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,514,7.6,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1702,10.0,0.015309,0.0,0.179637,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,365,2.7,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
alg=LogisticRegression()
alg.fit(tf_train_vect,y_train)
alg.score(tf_train_vect,y_train)
b=alg.predict(tf_test_vect)
count1=0
count0=1
for i in b:
    if i==1:
        count1+=1
    else:
        count0+=1

C:\Users\varun\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1852

In [70]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
cm=confusion_matrix(y_test,b)
print(cm)
print(accuracy_score(y_test,b))
print(classification_report(y_test,b))

[[1711    2]
 [ 140  439]]
0.9380453752181501
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1713
           1       1.00      0.76      0.86       579

    accuracy                           0.94      2292
   macro avg       0.96      0.88      0.91      2292
weighted avg       0.94      0.94      0.94      2292

